# Recherche d'information : librairie PyTerrier

Dans cette partie, nous nous intéressons à la librairie [PyTerrier](https://pyterrier.readthedocs.io/en/latest/#) qui permet de mettre en place diverses briques d'un moteur de recherche.
PyTerrier est basée sur [Terrier](http://terrier.org/) qui est un moteur de recherche développé en Java.

Nous allons voir :

- l'installation et la configuration
- l'indexation d'une collection
- l'accès à l'index
- l'évaluation d'un moteur de recherche


## Installation ete configuration

Après l'installation de la librairie, il est nécessaire d'initialiser PyTerrier pour importer les fichiers jar et démarrer la machine virtuelle associée;


## Indexation d'une collection

Il est possible d'indexer plusieurs formats de collection : format TREC, fichiers en texte brut ou en PDF, ou encore des Dataframe Pandas ([pour plus de détails](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html)).

Un petit exemple à titre illustratif est fourni dans le code suivant :


De nombreux fichiers sont créés : index direct, index inverse, méta-données de l'index et de la configuration de l'indexation, etc...

Il est également possible de modifier la configuration de l'indexation : [voir ici](https://pyterrier.readthedocs.io/en/latest/terrier-indexing.html#indexing-configuration).

Pour chager un index existant en local :


In [ ]:
index = pt.IndexFactory.of(index_ref)


Il est aussi possible de voir les statistiques de l'index.
pour connaître toutes les fonctions d'interrogation, se référencer à la [javadoc](http://terrier.org/docs/current/javadoc/org/terrier/structures/Index.html).


In [ ]:
# statistiques de la collection
print(index.getCollectionStatistics().toString())


Number of documents: 3
Number of terms: 4
Number of postings: 6
Number of fields: 0
Number of tokens: 7
Field names: []
Positions:   false



In [ ]:
# statistiques du vocabulaire.
# Nt : document frequency : nombre de documents contenant le terme
# TF : term frequency : nombre d'occurences du terme
# maxTF : nombre d'occurences maximal pour un document
for kv in index.getLexicon():
    print(
        "%s (%s) -> %s (%s)"
        % (
            kv.getKey(),
            type(kv.getKey()),
            kv.getValue().toString(),
            type(kv.getValue()),
        )
    )


document (<class 'str'>) -> term0 Nt=3 TF=4 maxTF=2 @{0 0 0} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
first (<class 'str'>) -> term1 Nt=1 TF=1 maxTF=1 @{0 0 7} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
topic (<class 'str'>) -> term2 Nt=1 TF=1 maxTF=1 @{0 1 1} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)
unknown (<class 'str'>) -> term3 Nt=1 TF=1 maxTF=1 @{0 1 5} (<class 'jnius.reflect.org.terrier.structures.LexiconEntry'>)


In [ ]:
# focus sur un terme particulier
index.getLexicon()["document"].toString()


'term0 Nt=3 TF=4 maxTF=2 @{0 0 0}'

In [ ]:
# récupère les statistiques de l'index inverse à partir d'un terme particulier
pointer = index.getLexicon()["document"]
for posting in index.getInvertedIndex().getPostings(pointer):
    print(posting.toString() + " doclen=%d" % posting.getDocumentLength())


ID(0) TF(2) doclen=3
ID(1) TF(1) doclen=1
ID(2) TF(1) doclen=3


De plus, PyTerrier met à disposition [une collection de jeux de données pré-traités](https://pyterrier.readthedocs.io/en/latest/datasets.html).
Dans ce qui suit, nous allons nous concentrer sur le jeu de données CORD19 qui recense des articles liés à la crise sanitaire Covid-19. Il est


**Exercice 1**

Afficher les statistiques de l'index Cord19 et analyser statistiques du terme "tv" (pas trop fréquent pour question d'affichage).


In [ ]:
# statistiques de la collection
print(index.getCollectionStatistics().toString())


Number of documents: 192509
Number of terms: 151235
Number of postings: 11554033
Number of fields: 1
Number of tokens: 17728468
Field names: [abstract]
Positions:   false



In [ ]:
index.getLexicon()["tv"].toString()


'term18759 Nt=70 TF=180 maxTF=2147483647 @{0 18614194 0} TFf=180'

## Recherche de documents à partir de l'index

Pour effectuer une recherche dans l'index, il faut utiliser la fonction batchRetrieve qui prend en paramètre l'index et le modèle de pondération (tf, tf-idf, etc...). La liste des modèles supportés est disponible [ici](http://terrier.org/docs/current/javadoc/org/terrier/matching/models/package-summary.html).


In [ ]:
br = pt.BatchRetrieve(index, wmodel="Tf")
br.search("chemical reactions")


,qid,docid,docno,rank,score,query
0,1,12409,ij3ncdb6,0,21.0,chemical reactions
1,1,24432,094bk0t0,1,13.0,chemical reactions
2,1,11268,qroxmo85,2,12.0,chemical reactions
3,1,120041,pb00yr0r,3,12.0,chemical reactions
4,1,147193,ei4rb8fr,4,12.0,chemical reactions
...,...,...,...,...,...,...
995,1,96252,8clnrlec,995,2.0,chemical reactions
996,1,96758,rvw94nsg,996,2.0,chemical reactions
997,1,97138,lrx94c81,997,2.0,chemical reactions
998,1,98181,w3160mgp,998,2.0,chemical reactions


On récupère alors un DataFrame dont les colonnes sont les suivantes :

- `qid` : identifiant de la requête. Par défaut, il s'agit de "1", puisqu'il s'agit de notre première et unique requête.
- `docid` : l'identifiant interne de Terrier pour chaque document
- `docno` : l'identifiant unique externe (chaîne de caractères) pour chaque document
- `score` : score des documents selon le modèle choisi (ici : fréquence totale des tf des termes de la requête dans chaque document)
- `rank` : rang du document dénotant l'ordre décroissant par score.
- `query` : la requête d'entrée


In [ ]:
# autre exemple de modèle : TF-IDF
tfidf = pt.BatchRetrieve(index, wmodel="TF_IDF")
tfidf.search("chemical reactions")


,qid,docid,docno,rank,score,query
0,1,18717,iavwkdpr,0,11.035982,chemical reactions
1,1,171636,v3blnh02,1,10.329726,chemical reactions
2,1,147193,ei4rb8fr,2,10.317138,chemical reactions
3,1,121217,msdycum2,3,9.653734,chemical reactions
4,1,170863,sj8i9ss2,4,9.500211,chemical reactions
...,...,...,...,...,...,...
995,1,2428,38aabxh1,995,3.790183,chemical reactions
996,1,14752,u709r8ss,996,3.790183,chemical reactions
997,1,20074,wxi1xsbo,997,3.790183,chemical reactions
998,1,117156,ts3obwts,998,3.790183,chemical reactions


On peut aussi fournir plusieurs requêtes grâce à un dataFrame. Pour interroger l'index, on applique la fonction transform() au BatchRetriever (br).
pour plus de détails, voir [les propriétés des transformations](https://pyterrier.readthedocs.io/en/latest/transformer.html) ainsi que les [opérations possibles](https://pyterrier.readthedocs.io/en/latest/operators.html).


In [ ]:
import pandas as pd

queries = pd.DataFrame(
    [["q1", "document"], ["q2", "first document"]], columns=["qid", "query"]
)
br.transform(queries)  # ou aussi : br(queries)


,qid,docid,docno,rank,score,query
0,q1,14569,mm7gxjf1,0,12.0,document
1,q1,137615,duxm9u8v,1,12.0,document
2,q1,18789,x3o3a45b,2,10.0,document
3,q1,189505,hxp258y8,3,10.0,document
4,q1,37445,q5wglpoj,4,9.0,document
...,...,...,...,...,...,...
1995,q2,137347,xye8pgzt,995,3.0,first document
1996,q2,137487,k0y0fz7k,996,3.0,first document
1997,q2,138448,ut3ayexx,997,3.0,first document
1998,q2,138807,fulh7trp,998,3.0,first document


**Exercice 2**

Ordonnancer les documents pour 3 requêtes : "covid disease", "hospital" et "home".
La fonction d'ordonnacement devra être de la forme suivante :

```
0.4 * score_Bm25 + 0.6 * score_Dirichlet
```


In [ ]:
queries = pd.DataFrame(
    [["q1", "covid disease"], ["q2", "hospital"], ["q3", "home"]],
    columns=["qid", "query"],
)

br_BM25 = pt.BatchRetrieve(index, wmodel="BM25")
br_Dirichlet = pt.BatchRetrieve(index, wmodel="DirichletLM")

res = ((0.4 * br_BM25 + 0.6 * br_Dirichlet) % 1000).transform(queries)
res


,qid,docid,docno,score,query,rank
0,q1,43498.0,9d8go2sl,3.067285,covid disease,1
1,q1,106017.0,a0w3n30z,3.067285,covid disease,2
2,q1,106018.0,smyp9fyo,3.067285,covid disease,3
3,q1,106019.0,vtfn9y5p,3.067285,covid disease,4
4,q1,158611.0,30ff3xg2,3.370553,covid disease,0
...,...,...,...,...,...,...
995,q1,130974.0,tm5cg9c1,2.159137,covid disease,997
996,q1,77346.0,f2m75jzk,2.649932,covid disease,244
997,q1,74650.0,hrbeh0me,2.613520,covid disease,295
998,q1,29690.0,5gvpjxxp,2.159012,covid disease,998


**Excercice 3**

Créez un ordonnanceur qui effectue les opérations suivantes :

- obtient les 10 documents les mieux notés par fréquence de terme (wmodel="Tf")
- obtenir les 10 documents les mieux notés par TF.IDF (wmodel="TF_IDF")
- ré-ordonne uniquement les documents trouvés dans les DEUX paramètres de recherche précédents en utilisant BM25.

Combien de documents sont récupérés par ce pipeline complet pour la requête "chemical"?

Vérification : le document avec le docno "37771" devrait avoir un score de 12.426309 pour la requête "chemical".


In [ ]:
def ordonnanceur(query):
    br_Tf = pt.BatchRetrieve(index, wmodel="Tf") % 10
    br_TF_IDF = pt.BatchRetrieve(index, wmodel="TF_IDF") % 10
    br_BM25 = pt.BatchRetrieve(index, wmodel="BM25") % 10

    res_tf = br_Tf.transform(query)
    res_TF_IDF = br_TF_IDF.transform(query)
    res_BM25 = br_BM25.transform(query)

    res1 = res_tf.merge(res_TF_IDF, on=["qid", "docno"])
    return res1.merge(res_BM25, on=["qid", "docno"])


query = pd.DataFrame([["q1", "chemical"]], columns=["qid", "query"])
ordonnanceur(query)


,qid,docid_x,docno,rank_x,score_x,query_x,docid_y,rank_y,score_y,query_y,docid,rank,score,query
0,q1,37771,jn5qi1jb,0,9.0,chemical,37771,0,6.809923,chemical,37771,0,12.426309,chemical
1,q1,134305,0smev8vt,1,8.0,chemical,134305,2,6.736806,chemical,134305,2,12.292890,chemical
2,q1,2524,ifebw24e,2,7.0,chemical,2524,9,6.269341,chemical,2524,9,11.439890,chemical
3,q1,142104,77c9ohxj,4,7.0,chemical,142104,3,6.700190,chemical,142104,3,12.226076,chemical
4,q1,56631,sps45fj5,9,6.0,chemical,56631,6,6.380524,chemical,56631,6,11.642770,chemical


## Reformulation de requêtes

Il est également possible de mettre en place des pipelines de [reformulation de requêtes](https://pyterrier.readthedocs.io/en/latest/rewrite.html).


In [ ]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
dph = pt.BatchRetrieve(index, wmodel="DPH")
pipeline = dph >> bo1 >> dph
pipeline.search("chemical reactions")


,qid,docid,docno,rank,score,query_0,query
0,1,147193,ei4rb8fr,0,20.728032,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
1,1,171636,v3blnh02,1,20.684654,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
2,1,18717,iavwkdpr,2,11.160010,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
3,1,20409,1g9kmpdi,3,10.702739,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
4,1,170863,sj8i9ss2,4,10.347593,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
...,...,...,...,...,...,...,...
995,1,114233,2cxb65j1,995,3.942073,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
996,1,183279,8emqwp2o,996,3.942073,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
997,1,88197,f7g1zacc,997,3.941175,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...
998,1,178340,zsp5v8hv,998,3.938286,chemical reactions,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...


L'autre solution est de l'intégrer directement dans la fonction d'ordonnancement. Mais la requête reformulée n'est pas visible et la solution précédente fait prendre conscience de la pipeline faite par le système de RI (ranking >> reformulation >> ranking quand on utilise des modèles basés sur la relevance feedback. ou reformulation >> ranking sinon). Plus d'exemples [ici](https://pyterrier.readthedocs.io/en/latest/rewrite.html).


In [ ]:
# modèle DPH avant reformulation de requête
pipelineQE = pt.BatchRetrieve(
    index, wmodel="DPH", controls={"qemodel": "Bo1", "qe": "off"}
)
pipelineQE.search("chemical reactions")


,qid,docid,docno,rank,score,query
0,1,171636,v3blnh02,0,11.898228,chemical reactions
1,1,147193,ei4rb8fr,1,11.889920,chemical reactions
2,1,18717,iavwkdpr,2,11.688030,chemical reactions
3,1,170863,sj8i9ss2,3,10.143884,chemical reactions
4,1,121217,msdycum2,4,9.860973,chemical reactions
...,...,...,...,...,...,...
995,1,154582,q4neat3x,995,4.070840,chemical reactions
996,1,171895,yjbcvf3o,996,4.070840,chemical reactions
997,1,14781,0it25j3k,997,4.067507,chemical reactions
998,1,70003,a1nc5k51,998,4.067507,chemical reactions


In [ ]:
# modèle DPH après reformulation de requête
pipelineQE = pt.BatchRetrieve(
    index, wmodel="DPH", controls={"qemodel": "Bo1", "qe": "on"}
)
pipelineQE.search("chemical reactions")


,qid,docid,docno,rank,score,query
0,1,147193,ei4rb8fr,0,20.728032,chemical reactions
1,1,171636,v3blnh02,1,20.684654,chemical reactions
2,1,18717,iavwkdpr,2,11.160010,chemical reactions
3,1,20409,1g9kmpdi,3,10.702739,chemical reactions
4,1,170863,sj8i9ss2,4,10.347593,chemical reactions
...,...,...,...,...,...,...
995,1,114233,2cxb65j1,995,3.942073,chemical reactions
996,1,183279,8emqwp2o,996,3.942073,chemical reactions
997,1,88197,f7g1zacc,997,3.941175,chemical reactions
998,1,178340,zsp5v8hv,998,3.938286,chemical reactions


### Evaluation d'un système de recherche d'information

Pour évaluer un système de RI, il est nécessaire d'avoir un jeu de données constitué de requêtes et de jugements de pertinence.


In [ ]:
# exemple de jugements de pertinence pour les 5 premières requêtes
cord19.get_qrels().head(5)


,qid,docno,label,iteration
0,1,005b2j4b,2,4.5
1,1,00fmeepz,1,4
2,1,010vptx3,2,0.5
3,1,0194oljo,1,2.5
4,1,021q9884,1,4


**Exercice 4**

A partir des requêtes et des jugements de pertinence du jeu de données CORD19, écrire le code qui permet d'afficher les résultats de la première requête de Cord19. L'affichage fusionnera les colonnes retournées par le BatchRetriever et les colonnes des qrels (merge sur qid et docno pour rajouter label et iteration au tableau).


Il existe cependant une fonction qui permet de calculer l'efficacité de ces ordonnancements au travers des métriques d'évaluation (map, précision, rappel, ndcg, ...)


In [ ]:
pt.Experiment(
    [tfidf],
    cord19.get_topics(variant="title"),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"],
)


,name,map,ndcg
0,BR(TF_IDF),0.180002,0.370767


**Exercice 5**

Réaliser une expérience comparant l'expansion de requêtes avec le modèle Bo1 et basé sur la KL-divergence; L'expérience est réalisée sur TREC CORD19 avec le modèle de référence BM25. Vous devrez construire des pipelines appropriées (plus de détails sur [l'expansion](https://pyterrier.readthedocs.io/en/latest/rewrite.html) et les [expérimentations](https://pyterrier.readthedocs.io/en/latest/experiments.html)).

Quelles approches entraînent des augmentations significatives de NDCG et MAP ou autres métriques ?


In [ ]:
bo1 = pt.rewrite.Bo1QueryExpansion(index)
kl = pt.rewrite.KLQueryExpansion(index)
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

pipeline1 = bm25 >> bo1 >> bm25 >> kl >> bm25
pipeline1.search("chemical reactions").head(5)

,qid,docid,docno,rank,score,query_0,query_1,query
0,1,147193,ei4rb8fr,0,41.700393,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.254141554 hazard^0.531991272 reactiv^0.505936553 explo^0.168494701 exotherm^0.367119086 bioprocess...
1,1,171636,v3blnh02,1,41.566071,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.254141554 hazard^0.531991272 reactiv^0.505936553 explo^0.168494701 exotherm^0.367119086 bioprocess...
2,1,20409,1g9kmpdi,2,19.007163,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.254141554 hazard^0.531991272 reactiv^0.505936553 explo^0.168494701 exotherm^0.367119086 bioprocess...
3,1,18717,iavwkdpr,3,18.734240,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.254141554 hazard^0.531991272 reactiv^0.505936553 explo^0.168494701 exotherm^0.367119086 bioprocess...
4,1,170863,sj8i9ss2,4,18.290762,applypipeline:off chemic^1.457741865 reaction^1.302919515 hazard^0.306805986 reactiv^0.283917121 explos^0.245621780 exotherm^0.236822495 bioproces...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.254141554 hazard^0.531991272 reactiv^0.505936553 explo^0.168494701 exotherm^0.367119086 bioprocess...


In [ ]:
pipeline2 = bm25 >> kl >> bm25 >> bo1 >> bm25
pipeline2.search("chemical reactions").head(5)

,qid,docid,docno,rank,score,query_0,query_2,query_1,query
0,1,147193,ei4rb8fr,0,52.077956,applypipeline:off chemic^1.457741865 reaction^1.195986382 hazard^0.517886470 reactiv^0.488200790 explo^0.147241157 exotherm^0.371181905 bioprocess...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.360348526 hazard^0.321524664 reactiv^0.311171535 explos^0.224282524 exotherm^0.204660628 bioproces...,applypipeline:off chemic^1.500948404 reaction^1.217459660 hazard^0.742454692 reactiv^0.708831574 explo^0.101006331 exotherm^0.502840124 bioprocess...
1,1,171636,v3blnh02,1,51.885147,applypipeline:off chemic^1.457741865 reaction^1.195986382 hazard^0.517886470 reactiv^0.488200790 explo^0.147241157 exotherm^0.371181905 bioprocess...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.360348526 hazard^0.321524664 reactiv^0.311171535 explos^0.224282524 exotherm^0.204660628 bioproces...,applypipeline:off chemic^1.500948404 reaction^1.217459660 hazard^0.742454692 reactiv^0.708831574 explo^0.101006331 exotherm^0.502840124 bioprocess...
2,1,20409,1g9kmpdi,2,24.725363,applypipeline:off chemic^1.457741865 reaction^1.195986382 hazard^0.517886470 reactiv^0.488200790 explo^0.147241157 exotherm^0.371181905 bioprocess...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.360348526 hazard^0.321524664 reactiv^0.311171535 explos^0.224282524 exotherm^0.204660628 bioproces...,applypipeline:off chemic^1.500948404 reaction^1.217459660 hazard^0.742454692 reactiv^0.708831574 explo^0.101006331 exotherm^0.502840124 bioprocess...
3,1,32633,iqnj1zfu,3,20.428426,applypipeline:off chemic^1.457741865 reaction^1.195986382 hazard^0.517886470 reactiv^0.488200790 explo^0.147241157 exotherm^0.371181905 bioprocess...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.360348526 hazard^0.321524664 reactiv^0.311171535 explos^0.224282524 exotherm^0.204660628 bioproces...,applypipeline:off chemic^1.500948404 reaction^1.217459660 hazard^0.742454692 reactiv^0.708831574 explo^0.101006331 exotherm^0.502840124 bioprocess...
4,1,94270,68701iad,4,20.428426,applypipeline:off chemic^1.457741865 reaction^1.195986382 hazard^0.517886470 reactiv^0.488200790 explo^0.147241157 exotherm^0.371181905 bioprocess...,chemical reactions,applypipeline:off chemic^1.523232555 reaction^1.360348526 hazard^0.321524664 reactiv^0.311171535 explos^0.224282524 exotherm^0.204660628 bioproces...,applypipeline:off chemic^1.500948404 reaction^1.217459660 hazard^0.742454692 reactiv^0.708831574 explo^0.101006331 exotherm^0.502840124 bioprocess...


In [ ]:
pt.Experiment(
    [pipeline1, pipeline2, bm25, tfidf],
    cord19.get_topics(variant="title"),
    cord19.get_qrels(),
    eval_metrics=["map", "ndcg"],
)


,name,map,ndcg
0,"Compose(Compose(Compose(Compose(BR(BM25), QueryExpansion(/home/aymeric/Documents/DAC/S2/RITAL/RI/TME2/terrier_cord19/data.properties,3,10,<org.ter...",0.186375,0.390620
1,"Compose(Compose(Compose(Compose(Compose(Compose(BR(BM25), QueryExpansion(/home/aymeric/Documents/DAC/S2/RITAL/RI/TME2/terrier_cord19/data.properti...",0.177511,0.381193
2,BR(BM25),0.181478,0.373328
3,BR(TF_IDF),0.180002,0.370767
